<a href="https://colab.research.google.com/github/Strikersid07/Projects/blob/main/Group_2_23_24_FI532E_Financial_Data_Infrastructure_Lending_Club.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#        FINANCIAL DATA INFRASTRUCTURE     
##                    Rennes School of Business
##                     Group Project Prediction of Default of Loan
####       LendingClub | Peer-to-Peer Lending & Alternative Investing
                                        Winter 2021





The following Paython code is Filter data given various criteria from a big data set of LendingClub.
This is a Peer-to-Peer Lending and Alternative Investing business. LendingClub is a US peer-to-peer lending company, headquartered in San Francisco, California. It was the first peer-to-peer lender to register its offerings as securities with the Securities and Exchange Commission (SEC), and to offer loan trading on a secondary market. LendingClub is the world's largest peer-to-peer lending platform. LendingClub enables borrowers to create unsecured personal loans between 1,000 and 40,000 dollars. The standard loan period is three years. Investors can search and browse the loan listings on LendingClub website and select loans that they want to invest in based on the information supplied about the borrower, amount of loan, loan grade, and loan purpose. Investors make money from interest. LendingClub makes money by charging borrowers an origination fee and investors a service fee.The dataset contain complete loan data for all loans issued through the 2007-2015, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. Additional features include credit scores, number of finance inquiries, address including zip codes, and state, and collections among others. Each row is divided by an individual loan id and member id, of course, for the interest of privacy each member id has been removed from the dataset.


For this task is to identify the most important variables to explain default of loans. To do this you need to develope model in the Spark framework and using textual infomation of description of loan lending club.


You can find information about dataset from LCDataDictionary.csv file uploaded to moodle.


Report

For this task, you need to pick seed number for sampling of data accourding to your group number, and forecast the news labes.


##Students name 	  Student No.	      
##1. KHARB Shantnu - ETU20230074
## 2. VERMA Ish - ETU20232180
## 3. SHARMA Siddharth - ETU20230076
## 4. BHARDWAJ SANJAY -ETU20232122
## 5. MISHRA Gourav - ETU20221420
##Group: 2


The dataset contain complete loan data for all loans issued through the 2007-2015, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. Additional features include credit scores, number of finance inquiries, address including zip codes, and state, and collections among others. Each row is divided by an individual loan id and member id, of course, for the interest of privacy each member id has been removed from the dataset.

* loan_status:
* loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
* funded_amnt: The total amount funded by investors for that loan at that point in time.
* term: The Number of payments on the loan. Values are in months and can be either 36 or 60.
* grade: LC assigned loan grade
* sub_grade: LC assigned loan subgrade
* home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
* verification_status: The status of the loan during the listing period. Values: APPROVED, NOT_APPROVED.
* purpose: A category provided by the borrower for the loan request. Values are: debt_consolidation, medical, home_improvement, renewable_energy, small_business, wedding, vacation, moving, house, car, major_purchase, credit_card, other
* emp_length: Employment length in months. Possible values are whole numbers from 0 and higher. Null indicates not employed.
* int_rate: Interest Rate on the loan
* annual_inc: The annual income provided by the borrower during registration.
* dti: The borrower's debt to income ratio, calculated using the monthly payments on the total debt obligations, excluding mortgage, divided by self-reported monthly income.
* inq_last_6mths: information from the bureau last 6 months
* pub_rec: it indicates risk and hurts your ability to qualify for credit, obligations that were not paid
* revol_bal: sometimes a full payment doesn’t (or can’t) happen, The amount of credit you have versus the amount of credit you use is a factor in your credit score
* revol_util: the amount of credit the borrower is using relative to all available revolving credit.
* total_acc: The total Number of credit lines currently in the borrower's credit file
* total_pymnt: The total payment currently
* total_pymnt_inv: The total payment currently of investment
* total_rec_prncp:
* total_rec_int:  interest of recollect total.

### Running Pyspark in Colab

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python.
Follow the steps to install the dependencies:

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark

# Install Spark NLP
! pip install --ignore-installed spark-nlp

openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2c3799e3b5171797058ad27b93aff07cbb7c6bd15c1e6ee9a857989fab2fbcbb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
#Imports

import sys
sys.path.append('../../')

import sparknlp

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import array_contains
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.3.3
Apache Spark version:  3.5.1


In [ ]:
from pyspark.ml.classification import OneVsRest
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.classification import RandomForestClassifier,LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.feature import HashingTF, IDF, StringIndexer, SQLTransformer,IndexToString

import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import NGram

In [ ]:
##Import csv files Survey and species to Spark
## It will take a long time to upload data

df_lending_club_loans = spark.read.format("csv").options(header='true', delimiter = ',').load("/content/Lending Club Loan Data_Lending Club - loan.csv")
# print structure of data
df_lending_club_loans.printSchema()
print(df_lending_club_loans.count())

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [ ]:
df_lending_club_loans.show(5)

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+--------------------+--------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------

##DATA MANUPULATION FROM SQL

In [ ]:
## Create a view to work with data
df_lending_club_loans.createOrReplaceTempView("club_loans")

In [ ]:
# Check first five observations of club_loans
spark.sql("select * from club_loans").show(3)

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+--------------------+--------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+---

In [ ]:
# Count number of loans by grouping loan_status
# Explain your findings (You can focus on Default)

spark.sql("select loan_status, count(*)\
 from club_loans group by loan_status order by loan_status").show()



#According to the data

#1. Charged Off: Of all the loans, 45,248 are designated as "Charged Off," making up roughly 5.10% of the total. "Charged Off" usually means that the debt has been written off as a loss by the lender since they believe it is unlikely to be repaid.
#2. Current: Of all the loans, 601,779 are classified as "Current," or roughly 67.82% of the total amount of loans. The "current" status denotes that the loan is being repaid in accordance with the conditions of the contract at this time.
#3. Default: Of all the loans, 1,219 are classified as "Default," or around 0.14% of the total. Loans that are in "Default" status have not been repaid, which puts the lender at greater risk of loss.
#4. Fully Paid: Of all loans, 207,723 are classified as "Fully Paid," making up roughly 23.41% of the total. The status "Fully Paid" denotes that the loan has been satisfactorily repaid in full by the borrower.
#5. Late (16-30 days) and Late (31-120 days): In total, 13,948 loans bear the designations "Late (16-30 days)" or "Late (31-120 days)," accounting for roughly 1.58% of all loans. These statuses show different degrees of delinquent in the borrower's missing payments.
#6. Other statuses: A lesser proportion of loans are in statuses like "Does not meet the credit policy," "In Grace Period," and "Issued," each of which accounts for a smaller portion of the overall number of loans.

+--------------------+--------+
|         loan_status|count(1)|
+--------------------+--------+
|         Charged Off|   45248|
|             Current|  601778|
|             Default|    1219|
|Does not meet the...|     761|
|Does not meet the...|    1988|
|          Fully Paid|  207723|
|     In Grace Period|    6253|
|              Issued|    8460|
|   Late (16-30 days)|    2357|
|  Late (31-120 days)|   11591|
|            Oct-2015|       1|
+--------------------+--------+



In [ ]:
# Group data by grade
#The credit rating of the customers is given by grade variable. The Lending Club divides customers'
#credit rating into 7 categories, e.g. A to G. The customers with a credit rating of A
#have the highest credit scores. The customers with a credit rating of G have the lowest ones. The customers with a high
#credit rating are less likely to default.
## Explain your findings

spark.sql("select grade, count(*)\
 from club_loans group by grade order by grade").show()

# Group data by sub_grade
spark.sql("select sub_grade, count(*)\
 from club_loans group by sub_grade").show()


#Grade Distribution: Each grade's total count is displayed for each loan in the dataset, which is grouped by grade. Grades are assigned on a scale from A to G, where "A" denotes the greatest credit rating and "G" the lowest. The counts for each grade are as follows:

#Grade A: 16421 loans
#Grade B: 25141 loans
#Grade C: 19705 loans
#Grade D: 12347 loans
#Grade E: 6105 loans
#Grade F: 2406 loans
#Grade G: 757 loans
#Null : 1 Loan
#According to these results, most loans are categorised as belonging to grades A, B, and C, with comparatively fewer loans falling into grades D, E, F, and G. Higher grades are indicative of lesser credit risk, which is consistent with the conventional grading system.


#Loan Status Distribution: The distribution of loans among various statuses is shown in the loan_status column. The following are important statuses: "Charged Off," "Current," "Default," "Fully Paid," and several delinquent statuses including "Late (16-30 days)" and "Late (31-120 days)." The counts for each status are as follows:

#Charged Off: 45,248 loans
#Current: 601,779 loans
#Default: 1,219 loans
#Fully Paid: 207,723 loans
#Others: Various other statuses with smaller counts

#These results shed light on the repayment patterns and loan performance within the sample. The fact that just a tiny portion of loans are in default, for example, is noteworthy and suggests generally strong repayment performance. But a substantial percentage of loans carry the designation "Charged Off," indicating that a sizable amount of loans have been written off as losses since they have not been repaid.


#Sub-grade Distribution: To provide a more detailed distribution within each grade, the dataset also includes loans classified by sub-grade. Within each grade, the sub-grade is a more detailed classification that reflects differences in credit risk. It displays the number of loans for each sub-grade. As an illustration:

#Sub-grade D5: 21,389 loans
#Sub-grade F2: 5,392 loans
#Sub-grade B4: 55,626 loans

#More in-depth understanding of the loan distribution within each grade is provided by this breakdown, which makes lending strategies and risk assessments more accurate.


+-----+--------+
|grade|count(1)|
+-----+--------+
|    A|  148202|
|    B|  254535|
|    C|  245860|
|    D|  139542|
|    E|   70705|
|    F|   23046|
|    G|    5489|
+-----+--------+

+---------+--------+
|sub_grade|count(1)|
+---------+--------+
|       D5|   21389|
|       F2|    5392|
|       B4|   55626|
|       A2|   22485|
|       E4|   11724|
|       B2|   48781|
|       C3|   50161|
|       D1|   36238|
|       C4|   48857|
|       F1|    7218|
|       D3|   26554|
|       F5|    2594|
|       G2|    1398|
|       B1|   44972|
|       B3|   56323|
|       E5|    9575|
|       C5|   41219|
|       G3|     981|
|       A4|   34531|
|       F4|    3409|
+---------+--------+
only showing top 20 rows



In [ ]:
# Provide the statistical description of annual income

spark.sql("SELECT count(*),AVG(annual_inc),\
  VARIANCE(annual_inc), MIN(annual_inc), MAX(annual_inc)\
  FROM club_loans").show()

+--------+----------------+--------------------+---------------+---------------+
|count(1)| avg(annual_inc)|variance(annual_inc)|min(annual_inc)|max(annual_inc)|
+--------+----------------+--------------------+---------------+---------------+
|  887379|75027.6294879158|4.1858732134060884E9|            0.0|       MORTGAGE|
+--------+----------------+--------------------+---------------+---------------+



In [ ]:
## Compare annual income variable with deflaut and non default status of the loan.

print("All loans")

spark.sql("SELECT count(*),AVG(annual_inc),\
  VARIANCE(annual_inc), MIN(annual_inc), MAX(annual_inc)\
  FROM club_loans").show()
print("Deflaut loans")
spark.sql("SELECT count(*),AVG(annual_inc),\
  VARIANCE(annual_inc), MIN(annual_inc), MAX(annual_inc)\
  FROM club_loans where loan_status = 'Default'").show()
print("Not Deflaut loans")
spark.sql("SELECT  count(*),AVG(annual_inc),\
  VARIANCE(annual_inc), MIN(annual_inc), MAX(annual_inc)\
  FROM club_loans where loan_status != 'Default'").show()

All loans
+--------+----------------+--------------------+---------------+---------------+
|count(1)| avg(annual_inc)|variance(annual_inc)|min(annual_inc)|max(annual_inc)|
+--------+----------------+--------------------+---------------+---------------+
|  887379|75027.6294879158|4.1858732134060884E9|            0.0|       MORTGAGE|
+--------+----------------+--------------------+---------------+---------------+

Deflaut loans
+--------+-----------------+--------------------+---------------+---------------+
|count(1)|  avg(annual_inc)|variance(annual_inc)|min(annual_inc)|max(annual_inc)|
+--------+-----------------+--------------------+---------------+---------------+
|    1219|66754.12661197703| 2.781379266540547E9|        10000.0|        98600.0|
+--------+-----------------+--------------------+---------------+---------------+

Not Deflaut loans
+--------+-----------------+--------------------+---------------+---------------+
|count(1)|  avg(annual_inc)|variance(annual_inc)|min(annual

In [ ]:
## Compare amount of loan variable with deflaut and non default status of the loan.
## Similar SQL code from pervious section
print("All loans")

spark.sql("SELECT COUNT(*), AVG(loan_amnt), VARIANCE(loan_amnt), MIN(loan_amnt), MAX(loan_amnt) FROM club_loans").show()

print("Default loans")
spark.sql("SELECT COUNT(*), AVG(loan_amnt), VARIANCE(loan_amnt), MIN(loan_amnt), MAX(loan_amnt) FROM club_loans WHERE loan_status = 'Default'").show()

print("Non-default loans")
spark.sql("SELECT COUNT(*), AVG(loan_amnt), VARIANCE(loan_amnt), MIN(loan_amnt), MAX(loan_amnt) FROM club_loans WHERE loan_status != 'Default'").show()



All loans
+--------+-----------------+-------------------+--------------+--------------+
|count(1)|   avg(loan_amnt)|variance(loan_amnt)|min(loan_amnt)|max(loan_amnt)|
+--------+-----------------+-------------------+--------------+--------------+
|  887379|14755.26460508982|7.115691120113865E7|        1000.0|        9975.0|
+--------+-----------------+-------------------+--------------+--------------+

Default loans
+--------+------------------+-------------------+--------------+--------------+
|count(1)|    avg(loan_amnt)|variance(loan_amnt)|min(loan_amnt)|max(loan_amnt)|
+--------+------------------+-------------------+--------------+--------------+
|    1219|15193.847415914684|6.915519610477778E7|        1000.0|        9975.0|
+--------+------------------+-------------------+--------------+--------------+

Non-default loans
+--------+------------------+-------------------+--------------+--------------+
|count(1)|    avg(loan_amnt)|variance(loan_amnt)|min(loan_amnt)|max(loan_amnt)|
+

In [ ]:
# Compare risk garde of Default loans
# Explain your finding
spark.sql("select grade, count(*) from club_loans\
 where loan_status = 'Default' group by grade order by count(*) DESC").show()



# Grade Distribution:
#
# Grade C has the highest count of default loans with 360 instances, followed closely by Grade D with 312 instances.
# Grades E and B also have significant default counts, with 201 and 198 instances respectively.
# Grades F, A, and G have lower counts of default loans, with 79, 47, and 22 instances respectively.
# Risk Assessment:
#
# The higher counts of default loans in Grades C and D suggest that loans in these categories are relatively riskier compared to others.
# Grades E and B also exhibit notable default counts, indicating a moderate level of risk associated with loans in these grades.
# Grades F, A, and G have fewer instances of default loans, implying that loans in these grades are comparatively less risky.
# Portfolio Management:
#
# Lenders or investors may use this information to adjust their portfolio strategies. For instance, they might consider diversifying investments away from higher-risk grades (C and D) or implementing stricter eligibility criteria for borrowers falling into these categories.
# Conversely, they might allocate more resources to Grade A loans, which have a lower likelihood of default based on the available data.
# Monitoring and Mitigation:
#
# These findings emphasize the importance of ongoing monitoring and risk mitigation measures for loans in higher-risk categories. Lenders may implement proactive measures such as early intervention strategies or adjusting interest rates to mitigate the impact of defaults.
# Predictive Modeling:
#
# The data on default loans across different grades can be used to train predictive models aimed at identifying potential defaulters at an early stage. By leveraging historical patterns and other relevant data, lenders can improve their risk assessment processes and make more informed lending decisions.
# In summary, the analysis of default loan distribution across various risk grades provides valuable insights for lenders and investors to optimize their portfolio management strategies, enhance risk assessment techniques, and mitigate potential losses associated with defaults.



+-----+--------+
|grade|count(1)|
+-----+--------+
|    C|     360|
|    D|     312|
|    E|     201|
|    B|     198|
|    F|      79|
|    A|      47|
|    G|      22|
+-----+--------+



In [ ]:
# Compare home_ownership of Default loans
# Explain your finding
## Similar SQL code from pervious section

spark.sql("SELECT home_ownership, COUNT(*) FROM club_loans WHERE loan_status = 'Default' GROUP BY home_ownership").show()




#1.Home Ownership Distribution among Default Loans: -
# Rent: Of the 611 loans in default, most borrowers rent their residences.
# Mortgage: A sizable portion of defaulted loans (498 loans) are owed by mortgage-holding borrowers.
# Own: Borrowers who own their houses account for a lesser percentage of default loans (110 loans).



#2. Interpretation: - Compared to homeowners, renters appear to be more frequently represented among defaulted loans. This may be a sign of a number of things, including unstable finances or increased danger while renting. On the other hand, mortgage holders also exhibit a significant presence in default loans, suggesting that even homeowners may experience difficulties repaying their debts. It's important to remember, too, that compared to renters and mortgage holders, the amount of default loans linked to homeowners who own their homes outright is quite low. This could imply that having a home of one's own could be a factor in lower default rates, but more research is required to be certain of this.

+--------------+--------+
|home_ownership|count(1)|
+--------------+--------+
|           OWN|     110|
|          RENT|     611|
|      MORTGAGE|     498|
+--------------+--------+



In [ ]:
# Compare home_ownership,grade,addr_state,verification_status, emp_length of Default loans
# Explain your finding
spark.sql("SELECT home_ownership, grade, addr_state, verification_status, emp_length,\
COUNT(*) FROM club_loans WHERE loan_status = 'Default' GROUP BY home_ownership, grade, addr_state, verification_status, emp_length").show()





#The provided SQL query output presents the distribution of default loans across various categories such as home ownership, grade, address state, verification status, and employment length. Here are the key findings:
#
#Home Ownership:
#Renters represent a significant portion of default loans, with various grades and employment lengths.
#Mortgage holders also contribute to default loans, although to a lesser extent compared to renters.
#
#
#Grade:
#Grades C and D are prominent among default loans across different home ownership statuses and address states.
#Grades A, B, E, and F are also present but to a lesser extent.
#
#
#Address State:
#Default loans are distributed across different states, with no single state dominating the default loan count.
#Further analysis may be required to identify any patterns or trends specific to certain states.
#
#
#Verification Status and Employment Length:
#Default loans are observed across various verification statuses and employment lengths.
#Interestingly, some default loans have missing or non-standard employment length values (e.g., 'n/a', '< 1 year'), indicating potential data quality issues.
#
#
#Interpretation:
#Renters and borrowers with lower-grade loans (C and D) seem to be more prone to default, suggesting higher risk associated with these groups.
#The presence of default loans across different states and employment lengths underscores the need for a comprehensive risk assessment strategy that considers multiple factors beyond a single criterion.

+--------------+-----+----------+-------------------+----------+--------+
|home_ownership|grade|addr_state|verification_status|emp_length|count(1)|
+--------------+-----+----------+-------------------+----------+--------+
|          RENT|    B|        CA|       Not Verified| 10+ years|       4|
|          RENT|    B|        NV|    Source Verified| 10+ years|       1|
|          RENT|    C|        OK|           Verified| 10+ years|       1|
|      MORTGAGE|    E|        IN|    Source Verified|   9 years|       1|
|          RENT|    C|        WI|       Not Verified|   2 years|       1|
|          RENT|    C|        AZ|    Source Verified|   7 years|       1|
|      MORTGAGE|    C|        CO|       Not Verified| 10+ years|       1|
|           OWN|    B|        NY|           Verified|  < 1 year|       1|
|      MORTGAGE|    C|        NY|       Not Verified|   7 years|       1|
|      MORTGAGE|    A|        NY|    Source Verified|    1 year|       1|
|          RENT|    C|        NY|     

In [ ]:
# Write your own SQL code to identify default loan

spark.sql("SELECT member_id,loan_amnt,funded_amnt,funded_amnt_inv, term,int_rate,installment,\
grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status, out_prncp,out_prncp_inv,\
total_pymnt,total_pymnt_inv,total_rec_prncp,last_pymnt_amnt FROM club_loans WHERE loan_status = 'Default' ORDER BY grade, addr_state, verification_status,emp_length, home_ownership").show()

+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+--------+-----------+---------+-------------+-----------+---------------+---------------+---------------+
|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status| issue_d|loan_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|last_pymnt_amnt|
+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+----------+--------------+----------+-------------------+--------+-----------+---------+-------------+-----------+---------------+---------------+---------------+
| 43712536|  15000.0|    15000.0|        15000.0| 36 months|    6.39|     458.99|    A|       A2|       n/a|      MORTGAGE|   49939.0|           Verified|Feb-2015|    Default| 12694.81|     12694.81|    2748.62|        2

#TEXT MINING

In [ ]:
# These are functions of preprocessing of text mining,
# Write few words about the process and functions

document_assembler = DocumentAssembler() \
    .setInputCol("desc") \
    .setOutputCol("document")# convert document to array of tokens
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

# clean tokens
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")# remove stopwords
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)# stems tokens to bring it to root form
stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")# Convert custom document structure to array of tokens.
finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)# To generate Term Frequency
ngram = NGram(n=3, inputCol="token_features", outputCol="token_features1")

hashingTF = HashingTF(inputCol="token_features1", outputCol="rawFeatures", numFeatures=1000)# To generate Inverse Document Frequency
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=100)# convert labels (string) to integers. Easy to process compared to string.
#label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")# define a simple Multinomial logistic regression model. Try different combination of hyperparameters and see what suits your data. You can also try different algorithms and compare the scores.
#label_to_stringIdx = IndexToString(inputCol="label", outputCol="article_class")# define the nlp pipeline


This code is a part of a machine learning pipeline that processes text data. It uses various techniques such as tokenization, normalization, stemming, and n-gramming to convert the text data into a numerical representation that can be fed into a machine learning model.
The code starts by defining a DocumentAssembler, which is a Spark MLlib class that converts a document into an array of tokens. The input column is "desc" and the output column is "document".
Next, it defines a Tokenizer, which is a Spark MLlib class that converts a document into an array of tokens. The input column is "document" and the output column is "token".
Then, it defines a Normalizer, which is a Spark MLlib class that removes stopwords from a tokenized document. The input column is "token" and the output column is "normalized".
After that, it defines a StopWordsCleaner, which is a Spark MLlib class that removes stopwords from a tokenized document. The input column is "normalized" and the output column is "cleanTokens".
Next, it defines a Stemmer, which is a Spark MLlib class that stems tokens to bring them to their root form. The input column is "cleanTokens" and the output column is "stem".
Finally, it defines a Finisher, which is a Spark MLlib class that converts the output of the stemmer into an array of tokens. The input column is "stem" and the output column is "token_features".
The code also defines an NGram, which is a Spark MLlib class that generates n-grams from a tokenized document. The input column is "token_features" and the output column is "token_features1".
The code also defines a HashingTF, which is a Spark MLlib class that converts the output of the NGram into a numerical representation. The input column is "token_features1" and the output column is "rawFeatures". The number of features is set to 1000.
Related

In [ ]:
nlp_pipeline = Pipeline(
    stages=[document_assembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            stemmer,
            finisher,
            ngram,
            hashingTF,
            idf])
            #label_stringIdx])

In [ ]:
model = nlp_pipeline.fit(df_lending_club_loans)
result = model.transform(df_lending_club_loans)
#result.show(10,False)
#result[['features']].show(10,False)
#result[['label']].show(10,False)
#result.show(2)
#data = result.select(['Date','features','Following'])

#dtt2 = result.select(['Date','features','Following'])#,'Following','Listed'])#,'rawFeatures'])
#na = data[data.'Followers'.isnull()]
#na
#dtt2.show(5)


In [ ]:
result.show(2)

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+--------------------+-----------+--------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------

In [ ]:
## you need to replace Na and Null values with zero

result=result.na.fill(value=0)


#DATA TRANSFORMATION

In [ ]:
data = result.select(['loan_status','features'])
data.show(5)
#Standard Sclarizer
#So we have created a feature vector. Now let us use StandardScaler to scalerize the newly created “feature” column
from pyspark.ml.feature import StandardScaler
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
data=standardscaler.fit(data).transform(data)
data.select("features","Scaled_features").show(5,False)

#from pyspark.ml.feature import ChiSqSelector
#css = ChiSqSelector(featuresCol='features', outputCol='Selected_features', labelCol='grade', fpr=0.05)
#data = css.fit(data).transform(data)
#data.select("features", "Selected_features").show(5, False)


+-----------+--------------------+
|loan_status|            features|
+-----------+--------------------+
| Fully Paid|(1000,[81,236,669...|
|Charged Off|(1000,[35,94,124,...|
| Fully Paid|        (1000,[],[])|
| Fully Paid|(1000,[141,208,26...|
|    Current|(1000,[18,94,116,...|
+-----------+--------------------+
only showing top 5 rows

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Define stages for encoding 'grade' column
string_indexer = StringIndexer(inputCol='loan_status', outputCol='loan_statusIndex', handleInvalid="skip")
one_hot_encoder = OneHotEncoder(inputCol='loan_statusIndex', outputCol='loan_status_encoded')

# Create encoding pipeline
encoding_pipeline = Pipeline(stages=[string_indexer, one_hot_encoder])
encoded_data = encoding_pipeline.fit(data).transform(data)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------+
|features                                                                                                                                                                                                                                           

#MACHINE LEARNIG MODEL FITTING

In [ ]:

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["indexed_features"], outputCol="assembled_features")

# Transform the data using the VectorAssembler
assembled_data = assembler.transform(indexed_data)

In [ ]:
## Create train and test sets
train, test = encoded_data.randomSplit([0.8, 0.2], seed=12345)

In [ ]:
#Model 1
### Run a Random Forest model with default parameters with orginal features
rf = RandomForestClassifier(labelCol="loan_statusIndex", featuresCol="Scaled_features", numTrees=10, maxDepth=10)
RFModel = rf.fit(train)
transformed = RFModel.transform(test)
transformed.show(2)
evaluator = MulticlassClassificationEvaluator(labelCol="loan_statusIndex", predictionCol="prediction",metricName="accuracy")

# compute the classification error on test data.
accuracy = evaluator.evaluate(transformed)
weighted_precision = evaluator.evaluate(transformed, {evaluator.metricName: "weightedPrecision"})
weighted_recall = evaluator.evaluate(transformed, {evaluator.metricName: "weightedRecall"})
print("Error = %g" % (1.0 - accuracy))
print("Accuracy = %g" % ( accuracy))
print ("Precision:", weighted_precision)
print ("Recall:", weighted_recall)

+-----------+------------+---------------+----------------+-------------------+--------------------+--------------------+----------+
|loan_status|    features|Scaled_features|loan_statusIndex|loan_status_encoded|       rawPrediction|         probability|prediction|
+-----------+------------+---------------+----------------+-------------------+--------------------+--------------------+----------+
|Charged Off|(1000,[],[])|   (1000,[],[])|             2.0|     (10,[2],[1.0])|[7.03162626626183...|[0.70316262662618...|       0.0|
|Charged Off|(1000,[],[])|   (1000,[],[])|             2.0|     (10,[2],[1.0])|[7.03162626626183...|[0.70316262662618...|       0.0|
+-----------+------------+---------------+----------------+-------------------+--------------------+--------------------+----------+
only showing top 2 rows

Error = 0.309338
Accuracy = 0.690662
Precision: 0.64796587855818
Recall: 0.6906619398688087


In [ ]:
#Model 2
### Run a logistic regression model with default parameters with orginal features


lr = LogisticRegression(labelCol="loan_statusIndex", featuresCol="Scaled_features",maxIter=10, regParam=0.3, elasticNetParam=0.0)# To convert index(integer) to corresponding class labels
lrModel=lr.fit(train)
result = lrModel.transform(test)
predict_test=lrModel.transform(test)
predict_test.select("loan_status","prediction").show(5)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(labelCol="loan_statusIndex", predictionCol="prediction",metricName="accuracy")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predict_test)
weighted_precision = evaluator.evaluate(predict_test, {evaluator.metricName: "weightedPrecision"})
weighted_recall = evaluator.evaluate(predict_test, {evaluator.metricName: "weightedRecall"})
print("Error = %g" % (1.0 - accuracy))
print("Accuracy = %g" % ( accuracy))
print ("Precision:", weighted_precision)
print ("Recall:", weighted_recall)



+-----------+----------+
|loan_status|prediction|
+-----------+----------+
|Charged Off|       0.0|
|Charged Off|       0.0|
|Charged Off|       0.0|
|Charged Off|       0.0|
|Charged Off|       0.0|
+-----------+----------+
only showing top 5 rows

Error = 0.292465
Accuracy = 0.707535
Precision: 0.6296604659076002
Recall: 0.7075350276043489
